In [4]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import BasicAer
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
)

In [16]:
class FSPTimeIndexform():
    """Quantum Optimization for the FSP tIME Index FORME"""
    def __init__(self,timeSpan : int,numberMachine : int,procTime:[],numberJobs : int)-> None :
        """
        Args : 
        timeSpan : the makespan value
        numberMachine : machine number
        numverJobs : job's number
        """
        self.timespan = timeSpan
        self.numberMachine = numberMachine
        self.numberJobs = numberJobs
        self.procTime = procTime
    #Version with only constraints    
    def to_quadratic_program_1(self,approach)-> QuadraticProgram:
        """
        Create the quadratic program of flow shop using index time forme
        approch can be :
         1 : Time threshold approach
         2 : Approximate expression approach
         3 : Makespan Decomposition 
        """
        Im ={}
        mdl = Model(name = "FSP_timeIndexOperation")
        n = self.numberJobs
        m = self.numberMachine
        ops = n*m #the total number of operations
        T = self.timespan
        #create binary variable xi_t for operation i starts at t
        x = {(i,t):mdl.binary_var(name= f"x_{i}_{t}") for i in range(ops) for t in range(T)}

        #constraint 1 : Each job must start only once 
        for i in range(ops):
            mdl.add_constraint(mdl.sum(x[(i,k)] for k in range(T))==1)

        #constraint 2 : No overlaping under the same machine and two jobs can't start at the same time
        for j in range(m):
             Im[j]= {j+i*m for i in range(n)}
        setE={
        (i,ti,k,tk)  for key, value in Im.items() 
        for i in value 
        for k in value 
        if k!=i 
        for ti in range(T) 
        for tk in range(T) 
        if  ti<=tk<ti+self.procTime[i]
        }
        for (i,ti,k,tk) in setE:
            mdl.add_constraint((x[(i,ti)] + (x[(k,tk)])<=1))

        # constraint 3 : operation order within a job must be the same
        for i in range(n):
             for k in range(i*m,(i+1)*m) :
                 for tk in range(T) :
                     for tk1 in range(T) :
                          if tk+self.procTime[k]>tk1 and k<(i+1)*m-1 : mdl.add_constraint((x[(k,tk)] + (x[(k+1,tk1)]))<=1)
        op = from_docplex_mp(mdl)                                
        return op
    
    def to_quadratic_program_2(self)-> QuadraticProgram :
        
        mdl = Model(name = "FSP_timeIndexOperation")
        n = self.numberJobs
        m = self.numberMachine
        ops = n*m #the total number of operations
        T = self.timespan
        # create binary variable xi_t for operation i starts at t
        x = {(i,m,t):mdl.binary_var(name= f"x_{i}_{m}_{t}") for i in range(n) for m in range(T) for t in range(T)}
        # constraints 
        for i in range(n): 
            for j in range(m):
                mdl.add_constraint(mdl.sum(x[(i,j,k)] for k in range(T))==1)
        
            
        op = from_docplex_mp(mdl)    
        return op

    def QUBO(self,approach)->QuadraticProgram:  
        conv = QuadraticProgramToQubo(approach)
        return conv.convert(self.to_quadratic_program(approach))
       
    #Version with threshold     
        
FSP = FSPTimeIndexform(2,2,[0,1,0,2],2)       
mdl=FSP.to_quadratic_program_2()
print(mdl.export_as_lp_string())

'''
qubo = 
op, offset = qubo.to_ising()
print("operator:")
print(op)
algorithm_globals.random_seed = 10
quantum_instance = QuantumInstance(
    BasicAer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
exact_result = exact.solve(qubo)
print(exact_result)
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

'''




\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: FSP_timeIndexOperation

Minimize
 obj:
Subject To
 c0: x_0_0_0 + x_0_0_1 = 1
 c1: x_0_1_0 + x_0_1_1 = 1
 c2: x_1_0_0 + x_1_0_1 = 1
 c3: x_1_1_0 + x_1_1_1 = 1

Bounds
 0 <= x_0_0_0 <= 1
 0 <= x_0_0_1 <= 1
 0 <= x_0_1_0 <= 1
 0 <= x_0_1_1 <= 1
 0 <= x_1_0_0 <= 1
 0 <= x_1_0_1 <= 1
 0 <= x_1_1_0 <= 1
 0 <= x_1_1_1 <= 1

Binaries
 x_0_0_0 x_0_0_1 x_0_1_0 x_0_1_1 x_1_0_0 x_1_0_1 x_1_1_0 x_1_1_1
End



NameError: name 'qubo' is not defined